Imports

In [ ]:
import sys
from pathlib import Path
import numpy as np
import matplotlib as mpl
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
from diffpy.utils.parsers.loaddata import loadData
from bg_mpl_stylesheet.bg_mpl_stylesheet import bg_mpl_style

Voltage label to use when plotting.

In [ ]:
D_ELABELS = dict(li="$E_{\mathrm{we}}\;\mathrm{vs.\;Li/Li^{+}\;[V]}$",
                 la="$E_{\mathrm{we}}\;\mathrm{vs.\;Na/Na^{+}\;[V]}$",
                 general="$V\;[\mathrm{V}]$",
                 )
ELABEL = D_ELABELS["li"]

Stating the file name of the iPython notebook.

In [ ]:
nb_name = "xy_overview_stack.ipynb"

Dictionary with plot settings.

In [ ]:
D_PLOT = dict(dpi=600,
              figsize=(12, 12),
              height_ratio=(1, 1, 0.5),
              fs_labels=20,
              fs_ticks=14,
              xmin_upper=1,
              xmax_upper=10,
              xmin_lower=10,
              xmax_lower=20,
              emin=1,
              emax=3,
              tlabel="$t\;[\mathrm{h}]$",
              elabel=ELABEL,
              )

Printing the dictionary with plot settings.

In [ ]:
print(f"{80*'-'}\nThe following plot settings were found in the top of the "
      f"{nb_name} file:")
for k in list(D_PLOT.keys()):
      if len(k) < 8:
            print(f"\t{k}\t\t{D_PLOT[k]}")
      else:
            print(f"\t{k}\t{D_PLOT[k]}")
print(f"{80*'-'}\nPlease change the settings and rerun the code if neccessary.")

Checking whether code is executed from iPython notebook or terminal to be able 
to exit code properly.

In [ ]:
def is_nb():
    shell = get_ipython().__class__.__name__
    if shell == "ZMQInteractiveShell":
        nb_bool = True
    else:
        nb_bool = False

    return nb_bool

Checking whether `data_upper` and `data_lower` folders for data files exist.

In [ ]:
data_upper_path = Path.cwd() / "data_upper"
data_lower_path = Path.cwd() / "data_lower"
data_paths = [data_upper_path, data_lower_path]
for p in data_paths:
    if not p.exists():
        p.mkdir()
        s = f"{80*'-'}\nA folder called '{p.name}' has been created.\n"
        s += f"Please put your data files there and rerun the code.\n{80*'-'}"
        print(s)
        if is_nb():
            exit(keep_kernel=True)
        else:
            exit()

Checking whether any data files are present in the `data_upper` folder.

In [ ]:
data_upper_files = list(data_upper_path.glob("*.*"))
if len(data_upper_files) == 0:
    s = f"{80*'-'}\nNo files were found in the '{data_upper_path.name}' folder."
    s += f"\nPlease put your data files there and rerun the code.\n{80*'-'}"
    print(s)
    if is_nb():
        exit(keep_kernel=True)
    else:
        exit()

Checking whether any data files are present in the `data_lower` folder.

In [ ]:
data_lower_files = list(data_lower_path.glob("*.*"))
if len(data_lower_files) == 0:
    s = f"{80*'-'}\nNo files were found in the '{data_lower_path.name}' folder."
    s += f"\nPlease put your data files there and rerun the code.\n{80*'-'}"
    print(s)
    if is_nb():
        exit(keep_kernel=True)
    else:
        exit()

Checking whether `data_echem` folder exists.

In [ ]:
data_echem_path = Path.cwd() / "data_echem"
if not data_echem_path.exists():
    data_echem_path.mkdir()
    s = f"{80*'-'}\nA folder called '{data_echem_path.name}' has been created."
    s += f"\nPlease put your data files there and rerun the code.\n{80*'-'}"
    print(s)
    if is_nb():
        exit(keep_kernel=True)
    else:
        exit()

Checking whether that a single file is present in the `data_echem` folder.

In [ ]:
data_echem_files = list(data_echem_path.glob("*.*"))
if len(data_echem_files) == 0:
    s = f"{80*'-'}\nNo files were found in the '{data_echem_path.name}' folder."
    s += f"\nPlease put your data file there and rerun the code.\n{80*'-'}"
    print(s)
    if is_nb():
        exit(keep_kernel=True)
    else:
        exit()
elif len(data_echem_files) > 1:
    s = f"{80*'-'}\nMore than one file was found in the "
    s += f"'{data_echem_path.name}' folder.\nPlease put your single data file "
    s += f"there and rerun the code.\n{80*'-'}"
    print(s)
    if is_nb():
        exit(keep_kernel=True)
    else:
        exit()
else:
    data_echem_file = data_echem_files[0]

Function for checking that only one file extension is present among the data 
files.

In [ ]:
def suffix_check(files):
    suffixes = []
    for f in files:
        if f.suffix not in suffixes:
            suffixes.append(f.suffix)
    if len(suffixes) > 1:
        print(f"{80*'-'}\nThe following file extensions were found in the "
            f"'{f.parent.name}' folder:")
        for suffix in suffixes:
            print(f"\t{suffix}")
        s = f"{80*'-'}\nPlease review the '{f.parent.name}' folder to ensure "
        s += f"that only one file extension is\npresent.\n{80*'-'}"
        print(s)
        if is_nb():
            exit(keep_kernel=True)
        else:
            exit()
    else:
        suffix = suffixes[0].strip(".")
    
    return suffix

Checking whether only one file extension is present among the data files for the
`data_upper` and `data_lower` folders.

In [ ]:
D_PLOT["suffix_upper"] = suffix_check(data_upper_files)
D_PLOT["suffix_lower"] = suffix_check(data_lower_files)

Updating dictionary with plot settings with labels according to the file 
extension of the data files.

In [ ]:
D_PLOT["xlabel"] = "scan number"
for e in ["upper", "lower"]:
    k = f"suffix_{e}"
    if D_PLOT[k] == "gr":
        D_PLOT[f"ylabel_{e}"] = "$r\;[\mathrm{\AA}]$"
        D_PLOT[f"cbarlabel_{e}"] = "$G\;[\mathrm{\AA}^{-2}]$"
    elif D_PLOT[k] == "fq":
        D_PLOT[f"ylabel_{e}"] = "$Q\;[\mathrm{\AA}^{-1}]$"
        D_PLOT[f"cbarlabel_{e}"] = "$F\;[\mathrm{\AA}^{-1}]$"
    else:
        D_PLOT[f"ylabel_{e}"] = "$Q\;[\mathrm{\AA}^{-1}]$"
        D_PLOT[f"cbarlabel_{e}"] = "$I\;[\mathrm{arb.\;u.}]$"

Collecting x array from first data file and stacking y arrays into one array.

In [ ]:
D_XY = {}
for e in [data_upper_files, data_lower_files]:
    for i, f in enumerate(e):
        data = loadData(f)
        x, y = data[:, 0], data[:, 1]
        if i == 0:
            array = y
        else:
            array = np.column_stack((array, y))
    D_XY[f.parent.name] = dict(x=x, array=array)

Collecting time and voltage for echem.

In [ ]:
DATA_ECHEM = np.loadtxt(data_echem_file)
D_EC = dict(time=DATA_ECHEM[:, 0], voltage=DATA_ECHEM[:, 1])

Function to obtain indices for min and max values of array.

In [ ]:
def get_indices(array, min_value, max_value):
    min_index, max_index = None, None
    for i, e in enumerate(array):
        if e >= min_value:
            min_index = i
            break
    for i, e in enumerate(array):
        if e >= max_value:
            max_index = i
            break
    if isinstance(min_index, type(None)):
        min_index = 0
    if isinstance(max_index, type(None)):
        max_index = len(array) - 1

    return min_index, max_index

Shaping x array and stacked y array.

In [ ]:
for e in ["upper", "lower"]:
    x, array = D_XY[f"data_{e}"]["x"], D_XY[f"data_{e}"]["array"]
    xmin_index, xmax_index = get_indices(D_XY[f"data_{e}"]["x"],
                                         D_PLOT[f"xmin_{e}"],
                                         D_PLOT[f"xmax_{e}"],
                                         )
    if xmax_index < len(x) - 1:
        x = x[xmin_index:xmax_index + 1]
        array = array[xmin_index:xmax_index + 1, :]
    else:
        x, array = x[xmin_index:], array[xmin_index:, :]
    D_XY[f"data_{e}"]["x"], D_XY[f"data_{e}"]["array"] = x, array
    D_PLOT[f"vmin_{e}"], D_PLOT[f"vmax_{e}"] = np.amin(array), np.amax(array)

Function for shifting (diverging) colormap, such thast white corresponds to zero
even if data is min and max is not symmetric around zero, i.e., if min != - max.

In [ ]:
def shiftedColorMap(cmap, start=0, midpoint=0.5, stop=1.0, name='shiftedcmap'):
    '''
    Function to offset the "center" of a colormap. Useful for
    data with a negative min and positive max and you want the
    middle of the colormap's dynamic range to be at zero.

    Input
    -----
      cmap : The matplotlib colormap to be altered
      start : Offset from lowest point in the colormap's range.
          Defaults to 0.0 (no lower offset). Should be between
          0.0 and `midpoint`.
      midpoint : The new center of the colormap. Defaults to
          0.5 (no shift). Should be between 0.0 and 1.0. In
          general, this should be  1 - vmax / (vmax + abs(vmin))
          For example if your data range from -15.0 to +5.0 and
          you want the center of the colormap at 0.0, `midpoint`
          should be set to  1 - 5/(5 + 15)) or 0.75
      stop : Offset from highest point in the colormap's range.
          Defaults to 1.0 (no upper offset). Should be between
          `midpoint` and 1.0.
    '''
    cdict = {
        'red': [],
        'green': [],
        'blue': [],
        'alpha': []
    }

    # regular index to compute the colors
    reg_index = np.linspace(start, stop, 257)

    # shifted index to match the data
    shift_index = np.hstack([
        np.linspace(0.0, midpoint, 128, endpoint=False),
        np.linspace(midpoint, 1.0, 129, endpoint=True)
    ])

    for ri, si in zip(reg_index, shift_index):
        r, g, b, a = cmap(ri)

        cdict['red'].append((si, r, r))
        cdict['green'].append((si, g, g))
        cdict['blue'].append((si, b, b))
        cdict['alpha'].append((si, a, a))

    newcmap = mcolors.LinearSegmentedColormap(name, cdict)
    mpl.colormaps.register(cmap=newcmap, force=True)

    return newcmap

Function to truncate (diverging) colormap to get the positive part.

In [ ]:
def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=100):
    new_cmap = mcolors.LinearSegmentedColormap.from_list(
               'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name,
                                                   a=minval,
                                                   b=maxval
                                                   ),
               cmap(np.linspace(minval, maxval, n)))

    return new_cmap

Creating diverging colormap with user-defined colors, shrinking colormap to 
adapt to data min and max values to have white corresponding to zero, and 
truncating the diverging colormap to get its positive part.

In [ ]:
div_gradient = mcolors.LinearSegmentedColormap.from_list('div_gradient', (
                 # Edit this gradient at 
                 # https://eltos.github.io/gradient/#0B3C5D-0B3C5D-FFFFFF-B82601-B82601
                 (0.000, (0.043, 0.235, 0.365)),
                 (0.250, (0.200, 0.400, 0.500)),
                 (0.500, (1.000, 1.000, 1.000)),
                 (0.750, (0.850, 0.200, 0.100)),
                 (1.000, (0.722, 0.149, 0.004))))
for e in ["upper", "lower"]:
    vmin, vmax = D_PLOT[f"vmin_{e}"], D_PLOT[f"vmax_{e}"]
    mp = 1 - vmax / (vmax + abs(vmin))
    D_PLOT[f"cmap_shrunk_{e}"] = shiftedColorMap(div_gradient, 
                                                 start=0, 
                                                 midpoint=mp, 
                                                 stop=1, 
                                                 name=f'shrunk_{e}',
                                                 )
    D_PLOT[f"cmap_trunc_{e}"] = truncate_colormap(div_gradient, 0.5, 1)

Updating dictionary with plot settings to include colormap.

In [ ]:
for e in ["upper", "lower"]:
    if D_PLOT[f"suffix_{e}"] in ["gr", "fq"]:
        D_PLOT[f"cmap_{e}"] = D_PLOT[f"cmap_shrunk_{e}"]
    else:
        D_PLOT[f"cmap_{e}"] = D_PLOT[f"cmap_trunc_{e}"]

Plot function.

In [ ]:
def plot(d_xy, d_ec, d, output_paths):
    plt.style.use(bg_mpl_style)
    fig = plt.figure(figsize=d["figsize"])
    grid = plt.GridSpec(nrows=3, 
                        ncols=2,
                        hspace=0.1,
                        height_ratios=d["height_ratio"], 
                        width_ratios=(1, 0.135),
                        )
    axs = [fig.add_subplot(grid[0, 0:]), 
           fig.add_subplot(grid[1, 0:]), 
           fig.add_subplot(grid[2, 0]),
           ]
    outputname = ""
    for i, k in enumerate(d_xy.keys()):
        e = k.split("_")[-1]
        x, array = d_xy[k]["x"], d_xy[k]["array"]
        im = axs[i].imshow(array,
                           interpolation="none",
                           aspect="auto",
                           extent=(0, 
                                   array.shape[1], 
                                   d[f"xmax_{e}"], 
                                   d[f"xmin_{e}"],
                                   ),
                           vmin=np.amin(array),
                           vmax=np.amax(array),
                           cmap=d[f"cmap_{e}"],
                           )
        if i == 0:
            axs[i].tick_params(axis="x",
                            top=True,
                            bottom=True,
                            labeltop=True,
                            labelbottom=False,
                            )
            axs[i].set_xlabel(d["xlabel"], fontsize=d["fs_labels"])
            axs[i].xaxis.set_label_position("top")
        else:
            axs[i].tick_params(axis="x",
                               top=True,
                               bottom=True,
                               labeltop=False,
                               labelbottom=False,
                               )  
        axs[i].tick_params(axis="y",
                           left=True,
                           right=True,
                           labelleft=True,
                           labelright=False,
                           )                  
        axs[i].set_ylabel(ylabel=d[f"ylabel_{e}"], fontsize=d["fs_labels"])
        axs[i].minorticks_on()
        cbar = plt.colorbar(im)
        cbar.set_label(label=d[f"cbarlabel_{e}"], fontsize=d["fs_labels"])
        if d[f"cbarlabel_{e}"] == "$I\;[\mathrm{arb.\;u.}]$":
            cbar.formatter.set_powerlimits((0, 0))
            cbar.ax.yaxis.set_offset_position('left')
            cbar.update_ticks()
        outputname += f"{d[f'suffix_{e}']}_"
    axs[-1].plot(d_ec["time"], d_ec["voltage"])
    axs[-1].set_xlim(np.amin(d_ec["time"]), np.amax(d_ec["time"]))
    axs[-1].set_ylim(d["emin"], d["emax"])
    axs[-1].set_xlabel(d["tlabel"], fontsize=d["fs_labels"])
    axs[-1].set_ylabel(d["elabel"], fontsize=d["fs_labels"])
    axs[-1].minorticks_on()
    
    outputname += f"overview_echem"
    for p in output_paths:
        print(f"\t{p.name}")
        plt.savefig(p / f"{outputname}.{p.name}", 
                    bbox_inches="tight",
                    dpi=d["dpi"],
                    )
    plt.close()

    return None

Creating plot directories if not already existing.

In [ ]:
plot_folders = ["png", "pdf", "svg"]
plot_paths = [Path.cwd() / folder for folder in plot_folders]
for p in plot_paths:
    if not p.exists():
        p.mkdir()

Plotting.

In [ ]:
print(f"{80*'-'}\nPlotting...")
plot(D_XY, D_EC, D_PLOT, plot_paths)
print(f"Done.\n{80*'-'}\nPlease see the {plot_folders} folders.")